In [58]:
import pandas as pd
import numpy as np
from scipy.stats import mode

In [59]:
# Load dataset
file_path = "../Interim/cleaned_food_prices.csv"
df = pd.read_csv(file_path)

In [60]:
# View the first few rows of the dataset
print(df.head(1))

       country                            Region Province  City   lat     lon  \
0  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   

         Date  year  month  beans  ...  l_tomatoes  c_tomatoes  \
0  2007-01-01  2007      1    NaN  ...       27.32       32.54   

   inflation_tomatoes  trust_tomatoes  o_food_price_index  h_food_price_index  \
0                 NaN             6.1                0.74                0.75   

   l_food_price_index  c_food_price_index  inflation_food_price_index  \
0                0.73                0.74                         NaN   

   trust_food_price_index  
0                     9.6  

[1 rows x 92 columns]


In [61]:
# Check the structure and datatypes
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24416 entries, 0 to 24415
Data columns (total 92 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       24416 non-null  object 
 1   Region                        24416 non-null  object 
 2   Province                      24416 non-null  object 
 3   City                          24416 non-null  object 
 4   lat                           24192 non-null  float64
 5   lon                           24192 non-null  float64
 6   Date                          24416 non-null  object 
 7   year                          24416 non-null  int64  
 8   month                         24416 non-null  int64  
 9   beans                         2770 non-null   float64
 10  cabbage                       4299 non-null   float64
 11  carrots                       4357 non-null   float64
 12  eggs                          3814 non-null   float64
 13  m

In [62]:
# Get summary statistics of numerical columns
print(df.describe())

                lat           lon          year         month        beans  \
count  24192.000000  24192.000000  24416.000000  24416.000000  2770.000000   
mean      11.709815    122.876852   2015.839286      6.428571    98.668827   
std        3.640444      1.949464      5.391164      3.437559    15.721549   
min        5.030000    118.740000   2007.000000      1.000000    56.600000   
25%        8.480000    121.090000   2011.000000      3.000000    87.500000   
50%       11.570000    122.855000   2016.000000      6.000000    99.210000   
75%       14.712500    124.667500   2020.250000      9.000000   108.750000   
max       18.190000    126.210000   2025.000000     12.000000   229.540000   

           cabbage      carrots         eggs  meat_beef_chops  \
count  4299.000000  4357.000000  3814.000000      4107.000000   
mean     73.586174    94.882415     7.173199       281.299333   
std      32.077402    39.001340     1.559628        72.732810   
min       1.600000    15.620000     4

In [63]:
#dropping columns related to food price index
df_nofpi = df.drop(columns=['o_food_price_index', 'h_food_price_index', 'l_food_price_index', 'c_food_price_index', 'inflation_food_price_index', 'trust_food_price_index'])

In [64]:
# Convert 'Date' column to datetime format
df_nofpi['Date'] = pd.to_datetime(df_nofpi['Date'])

In [65]:
# Check the unique values of the 'Date' column (like ranges)
print(df_nofpi['Date'].min(), df_nofpi['Date'].max())

2007-01-01 00:00:00 2025-08-01 00:00:00


In [66]:
#dropping columns related to inflation
df_noinf = df_nofpi.drop(columns=['inflation_beans','inflation_cabbage', 'inflation_carrots', 'inflation_eggs', 'inflation_meat_beef_chops', 'inflation_meat_chicken_whole', 'inflation_meat_pork', 'inflation_onions', 'inflation_potatoes', 'inflation_rice', 'inflation_tomatoes'])

In [67]:
#dropping columns related to trust scores
df_cleaned = df_noinf.drop(columns=['trust_beans','trust_cabbage', 'trust_carrots', 'trust_eggs', 'trust_meat_beef_chops', 'trust_meat_chicken_whole', 'trust_meat_pork', 'trust_onions', 'trust_potatoes', 'trust_rice', 'trust_tomatoes'])

In [68]:
# View the dataset after dropping columns
print(df_cleaned.head())

       country                            Region Province  City   lat     lon  \
0  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   
1  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   
2  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   
3  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   
4  Philippines  Cordillera Administrative region     Abra  Abra  17.6  120.62   

        Date  year  month  beans  ...  l_potatoes  c_potatoes  o_rice  h_rice  \
0 2007-01-01  2007      1    NaN  ...       36.02       38.25   19.19   19.58   
1 2007-02-01  2007      2    NaN  ...       36.34       36.90   19.35   19.67   
2 2007-03-01  2007      3    NaN  ...       34.54       36.57   19.80   20.25   
3 2007-04-01  2007      4    NaN  ...       34.64       36.30   20.43   20.77   
4 2007-05-01  2007      5    NaN  ...       34.58       37.20   19.76   20.63   

   l_rice  c_rice  o_tomat

In [69]:
#dropping uneeded columns
df_unneeded = df_cleaned.drop(columns=['country', 'City','Province', 'lat', 'lon'])

# Reshaping from wide to long format (including year and month as part of the identifier)
df_melted = df_unneeded.melt(id_vars=['Region', 'Date', 'year', 'month'], var_name='Food_Items', value_name='Price')

In [70]:
# View the dataset after reshaping
print(df_melted.head())

                             Region       Date  year  month Food_Items  Price
0  Cordillera Administrative region 2007-01-01  2007      1      beans    NaN
1  Cordillera Administrative region 2007-02-01  2007      2      beans    NaN
2  Cordillera Administrative region 2007-03-01  2007      3      beans    NaN
3  Cordillera Administrative region 2007-04-01  2007      4      beans    NaN
4  Cordillera Administrative region 2007-05-01  2007      5      beans    NaN


## Start of analysis

### 1.Exploratory Data Analysis

In [71]:
print(df_melted['Region'].unique())
print(df_melted['Food_Items'].unique())

['Cordillera Administrative region' 'Region XIII' 'Region VI' 'Region V'
 'Region III' 'Autonomous region in Muslim Mindanao' 'Region IV-A'
 'Region VIII' 'Region VII' 'Region X' 'Region II' 'Region IV-B'
 'Region XII' 'Region XI' 'Region I' 'National Capital region' 'Region IX'
 'Market Average']
['beans' 'cabbage' 'carrots' 'eggs' 'meat_beef_chops' 'meat_chicken_whole'
 'meat_pork' 'onions' 'potatoes' 'rice' 'tomatoes' 'o_beans' 'h_beans'
 'l_beans' 'c_beans' 'o_cabbage' 'h_cabbage' 'l_cabbage' 'c_cabbage'
 'o_carrots' 'h_carrots' 'l_carrots' 'c_carrots' 'o_eggs' 'h_eggs'
 'l_eggs' 'c_eggs' 'o_meat_beef_chops' 'h_meat_beef_chops'
 'l_meat_beef_chops' 'c_meat_beef_chops' 'o_meat_chicken_whole'
 'h_meat_chicken_whole' 'l_meat_chicken_whole' 'c_meat_chicken_whole'
 'o_meat_pork' 'h_meat_pork' 'l_meat_pork' 'c_meat_pork' 'o_onions'
 'h_onions' 'l_onions' 'c_onions' 'o_potatoes' 'h_potatoes' 'l_potatoes'
 'c_potatoes' 'o_rice' 'h_rice' 'l_rice' 'c_rice' 'o_tomatoes'
 'h_tomatoes' 'l_tomat

#### Measures of Central Tendency

In [72]:
grouped = df_melted.groupby(['Region', 'Food_Items', 'year', 'month', 'Date'])

##### mean and median

In [73]:
central_tendency = grouped['Price'].agg(['mean', 'median']).reset_index()
print(central_tendency)

                                      Region Food_Items  year  month  \
0       Autonomous region in Muslim Mindanao      beans  2007      1   
1       Autonomous region in Muslim Mindanao      beans  2007      2   
2       Autonomous region in Muslim Mindanao      beans  2007      3   
3       Autonomous region in Muslim Mindanao      beans  2007      4   
4       Autonomous region in Muslim Mindanao      beans  2007      5   
...                                      ...        ...   ...    ...   
221755                           Region XIII   tomatoes  2025      4   
221756                           Region XIII   tomatoes  2025      5   
221757                           Region XIII   tomatoes  2025      6   
221758                           Region XIII   tomatoes  2025      7   
221759                           Region XIII   tomatoes  2025      8   

             Date       mean  median  
0      2007-01-01        NaN     NaN  
1      2007-02-01        NaN     NaN  
2      2007-03-01 

##### mode

In [74]:
#calculate mode
def calculate_mode(series):
    # Drop NaN values
    valid_values = series.dropna()
    
    if valid_values.empty:  # If no valid values, return NaN
        return np.nan
    
    # Compute the mode
    result = mode(valid_values)
    
    # Handle cases where result.mode might be scalar
    if isinstance(result.mode, np.ndarray) and len(result.mode) > 0:
        return result.mode[0]
    elif np.isscalar(result.mode):  # For scalar mode
        return result.mode
    else:
        return np.nan

central_tendency['Mode'] = grouped['Price'].transform(calculate_mode)


In [75]:
print(central_tendency)

                                      Region Food_Items  year  month  \
0       Autonomous region in Muslim Mindanao      beans  2007      1   
1       Autonomous region in Muslim Mindanao      beans  2007      2   
2       Autonomous region in Muslim Mindanao      beans  2007      3   
3       Autonomous region in Muslim Mindanao      beans  2007      4   
4       Autonomous region in Muslim Mindanao      beans  2007      5   
...                                      ...        ...   ...    ...   
221755                           Region XIII   tomatoes  2025      4   
221756                           Region XIII   tomatoes  2025      5   
221757                           Region XIII   tomatoes  2025      6   
221758                           Region XIII   tomatoes  2025      7   
221759                           Region XIII   tomatoes  2025      8   

             Date       mean  median   Mode  
0      2007-01-01        NaN     NaN    NaN  
1      2007-02-01        NaN     NaN    NaN

#### Measures of Dispersion

##### Range

In [76]:
#dropping uneeded columns
df_unneeded = df_cleaned.drop(columns=['country', 'City', 'lat', 'lon', 'Region', 'Province', 'Date', 'month'])

# Reshaping from wide to long format (including year and month as part of the identifier)
df_melted = df_unneeded.melt(id_vars=['year'], var_name='Food_Items', value_name='Price')

grouped_range = df_melted.groupby(['Food_Items', 'year'])
range_df = grouped_range['Price'].agg(lambda x: x.max() - x.min()).reset_index()
range_df.rename(columns={'Price': 'Range'}, inplace=True)


# Save cleaned data to a CSV file
range_df.to_csv("range_results.csv", index=False)

##### Variance and Standard Deviation

In [77]:
dispersion_df = grouped['Price'].agg(['var', 'std']).reset_index()
dispersion_df.rename(columns={'var': 'Variance', 'std': 'Standard Deviation'}, inplace=True)

##### Interquartile Range

In [78]:
def compute_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    return q3 - q1

iqr_df = grouped['Price'].agg(compute_iqr).reset_index()
iqr_df.rename(columns={'Price': 'IQR'}, inplace=True)

##### combine all into one DataFrame

In [79]:
dispersion_merge_df = pd.merge(central_tendency, dispersion_df, on=['Region', 'Food_Items','Date', 'year', 'month'])
final_eda_df = pd.merge(dispersion_merge_df, iqr_df, on=['Region', 'Food_Items','Date', 'year', 'month'])

In [80]:
print(final_eda_df)

                                      Region Food_Items  year  month  \
0       Autonomous region in Muslim Mindanao      beans  2007      1   
1       Autonomous region in Muslim Mindanao      beans  2007      2   
2       Autonomous region in Muslim Mindanao      beans  2007      3   
3       Autonomous region in Muslim Mindanao      beans  2007      4   
4       Autonomous region in Muslim Mindanao      beans  2007      5   
...                                      ...        ...   ...    ...   
221755                           Region XIII   tomatoes  2025      4   
221756                           Region XIII   tomatoes  2025      5   
221757                           Region XIII   tomatoes  2025      6   
221758                           Region XIII   tomatoes  2025      7   
221759                           Region XIII   tomatoes  2025      8   

             Date       mean  median   Mode    Variance  Standard Deviation  \
0      2007-01-01        NaN     NaN    NaN         NaN 

In [81]:
# Save cleaned data to a CSV file
final_eda_df.to_csv("eda_results.csv", index=False)